# 0. Gathering the data

* Gathering √
* Assessing √
* Cleaning √
* Analyzing and visualizing √
* Reporting √

In [ ]:
# Libraries required for this notebook to run

import re
import json
import numpy as np

import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 800)

In [ ]:
# Upload archive and read the data into a pandas DataFrame

df_archive = pd.read_csv("twitter-archive-enhanced.csv")
df_archive.head()

In [ ]:
df_archive.shape

In [ ]:
# The HTTP 200 OK success status response code indicates that the request has succeeded 

url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)
response.status_code

In [ ]:
with open("image_predictions.tsv", mode='wb') as file:
    file.write(response.content)

In [ ]:
df_images_predictions = pd.read_csv("image_predictions.tsv", sep='\t')

In [ ]:
df_images_predictions.head()

In [ ]:
df_images_predictions.shape

In [ ]:
tweet_ids = df_archive.tweet_id.values
len(tweet_ids)

In [ ]:
df_json = pd.read_json("tweet_json.txt", lines=True, encoding = 'utf-8')
df_json.head()

In [ ]:
df_json = df_json[['id', 'favorite_count', 'retweet_count']]
df_json.head()

In [ ]:
df_json.columns

In [ ]:
df_json.rename(columns={"id": "tweet_id"}, inplace=True)

In [ ]:
# Step 1 in merging

df_1 = df_archive.copy()

In [ ]:
# Step 2 in merging

df_2 = df_1.merge(df_images_predictions, how='left', on='tweet_id')

In [ ]:
# Step 3 in merging 

df_3 = df_2.merge(df_json, how='left', on='tweet_id')

In [ ]:
# Step 4, the copy of dataframe in which all three dataframes are combined

df_all = df_3.copy()

# 1. Assessing the data

In [ ]:
df_all.duplicated().sum()

In [ ]:
df_all['tweet_id'].duplicated().sum()

In [ ]:
df_all.columns

In [ ]:
df_all.info()

In [ ]:
df_all.head()

In [ ]:
df_all.source.value_counts()

In [ ]:
df_all['in_reply_to_status_id'].sort_values(ascending=False)

In [ ]:
df_all.tail()

In [ ]:
df_all.sample(5)

In [ ]:
df_all.isnull().sum().sort_values(ascending=False)

In [ ]:
df_all.rating_numerator.value_counts()

In [ ]:
print(df_all.loc[df_all.rating_numerator == 3, 'text'])
print(df_all.loc[df_all.rating_numerator == 0, 'text']) 
print(df_all.loc[df_all.rating_numerator == 1776, 'text'])

In [ ]:
df_all.rating_denominator.value_counts()

In [ ]:
print(df_all.loc[df_all.rating_denominator == 11, 'text'])
print(df_all.loc[df_all.rating_denominator == 50, 'text'])
print(df_all.loc[df_all.rating_denominator == 0, 'text'])

In [ ]:
df_all['name'].value_counts(dropna=False)

In [ ]:
df_all['name'].sort_values

In [ ]:
df_all.loc[df_all['name'] == 'a'].sample(10)

In [ ]:
df_all.describe()

In [ ]:
df_all.p1.sample(25)

# 2. Cleaning the data

N.b. Instead of following the prescribed Define-Code-Test framework, I decided (for the sake of simplicity) to move the Test paragraph <u>before</u> the Code paragraph, so that instead of inserting the same line of code after every cleaning operation, all you have to do is scroll back up and run the same line of code, which will output an updated dataset after every subsequent cell.

## 2.1 Define

### 2.1.1 Tidiness issues
* 3 tables (*df_archive*, *df_images_predictions*, *df_json*) can be combined into 1 table (*df_all*) √
    * See "*Step 0: Gathering the data*"
* Combine the 4 Boolean dog stage columns into 1 object column called ‘*dog_stage*’ √

### 2.1.2 Quality issues
* The following columns can be removed from the dataset due to requirements and/or a lack of necessity to the analysis √:
    * '*in_reply_to_status_id*', '*in_reply_to_user_id*',  '*retweeted_status_id*', '*retweeted_status_user_id*’, '*retweeted_status_timestamp*’, ‘*expanded_urls*’, ‘*img_num*’, ‘*p2*’, ‘*p2_conf*’, ‘*p2_dog*’, ‘*p3*’, ‘*p3_conf*’, and ‘*p3_dog*’
* The Timestamp column’s datatype is **object** (Panda’s equivalent of string). Additionally, +0000 at the end of the timestamp is a reference to GMT, not the actual time. To maximize efficiency, this should be mentioned in the description of the variable, not in the data itsel √.
    * Convert timestamp from object to datetime
    * Remove +0000
* Part of the requirements is that the dataset should only contain tweets **including photos with original ratings**. Therefor, remove 181 retweets, remove (2356-2075=)281 tweets that contain no photo √.
* Replace the values in the ‘*source*’ column with only the name of the source and remove the html formatting for the sake of readability √.
* Inconsistent capitalization in image prediction column (p1) √.
* *tweet_id* is formatted as an integer, but variables should only be integers when one intends to perform calculations on them √.
* Rename various columns with ambigious names √.
* Identify and replace all values in the ‘name’ column that are not names: the algorithm selects the word following “*This is…*”. Assuming that the sentence (in this context) will never end after “*is*” and that only names are capitalized, replacing all non-names with “*None*” should solve this problem. Additionally, I decided to replace "words" of one character to remove 'a'. By sorting the values of the ‘*name*’ column and concluding that no entry starts with a number or some form of punctuation, this should suffice √.

#### 2.1.2.1 Additional issues identified after inspecting .csv in Excel
* The dog_stage column at the end did not make sense, so I moved it
* For some reason some of the confidence intervals had several zeros, which I removed for consistency

### 2.1.3 Order of cleaning operations
* Remove replies and retweets
* Remove timezone from timestamp and convert from *object* to *datetime*
* Drop columns (before inception of *dog_stage* and *shortened_url* to simplify .melt code)
* Extracted relevant information from *source* and replaced original values
* Inconsistent capitalization p1
* *tweet_id* data type
* Rename unclear column names
* Combine *doggo*, *floofer*, *pupper*, and *puppo* columns into a single column called *dog_stage*
* Identify and replace all values in the ‘name’ column that are not names
* Changed order of columns (dog_stage column)
* Standardized the amount of fraction digits to 6

In [ ]:
# A copy of the aggregated dataset to perform tests on that can be reiterated an infinite amount of times
df_clean = df_all.copy()

In [ ]:
# First five rows of aggregated dataset

df_clean.head()

In [ ]:
# Dimensions of aggregated dataset

df_clean.shape

## 2.2 Code and Test

### 2.2.1 Code (replies and retweets)

In [ ]:
# Remove replies
df_clean = df_clean[df_clean.in_reply_to_status_id.isnull()]

# Remove retweets
df_clean = df_clean[df_clean.retweeted_status_id.isnull()]

### 2.2.2 Test (replies and retweets)

In [ ]:
# Dimensions of aggregated dataset after removal of replies and 

df_clean.shape

### 2.3.1 Code (timestamp formatting)

In [ ]:
# Remove +0000 from timestamp column
df_clean.timestamp = df_clean.timestamp.str[:-6]

# Convert timestamp from object to datetime data structure
df_clean.timestamp = pd.to_datetime(df_clean.timestamp)

### 2.3.2 Test (timestamp formatting)

In [ ]:
# Sample of timestamp column after removal of GMT timezone and updated datatype

print(df_clean.timestamp.sample(1))

### 2.3.1 Code (removal of unnecessary columns)

In [ ]:
df_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id',  'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'img_num', 'jpg_url', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis=1, inplace=True)

### 2.3.2 Test (removal of unnecessary columns)

In [ ]:
print(df_clean.columns)

### 2.4.1 Code (modification of source column)

In [ ]:
# Output to be used in .replace command in next cell

df_clean.source.value_counts()

In [ ]:
# Clean up source column

df_clean.source = df_clean.source.replace(
    {'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>':'Twitter for iPhone',
    '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>':'Vine',
    '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>':'Twitter Web Client',
    '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>':'TweetDeck'})

### 2.4.2 Test (modification of source column)

In [ ]:
df_clean.source.value_counts()

### 2.5.1 Code (standardization of capitalization in p1 column)

In [ ]:
# Inconsistent capitalization p1
df_clean.p1 = df_clean.p1.str.lower()

### 2.5.2 Test (standardization of capitalization in p1 column)

In [ ]:
df_clean.p1.sample(10)

### 2.6.1 Code (data type of tweet_id column)

In [ ]:
# Dtype tweet_id
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

### 2.6.2 Test (data type of tweet_id column)

In [ ]:
df_clean.info()

### 2.7.1 Code (removal of tweets without a photo)

In [ ]:
# To identify the rows that do not contain a photo

df_clean.loc[~df_clean['text'].str.contains ('https', case=False, na=False)]

In [ ]:
# Remove tweets that do not contain a photo by index

df_clean.drop([375, 707, 1445], inplace=True)

### 2.7.2 Test (removal of tweets without a photo)

In [ ]:
# To identify the rows that do not contain a photo

df_clean.loc[~df_clean['text'].str.contains ('https', case=False, na=False)]

### 2.8.1 Code (column names)

In [ ]:
# Rename columns

df_clean.rename(columns = {'name':'dog_name','p1':'prediction','p1_conf':'CI_prediction','p1_dog':'is_dog'}, inplace = True)

### 2.8.2 Test (column names)

In [ ]:
df_clean.columns

### 2.9.1 Code (aggregation of type of dog columns)

In [ ]:
# Add dog_stage, remove additional four columns

df_clean = pd.melt(df_clean,
                  id_vars = ['tweet_id','timestamp','source','text','rating_numerator','rating_denominator','dog_name','prediction','CI_prediction','is_dog','favorite_count','retweet_count'],
                  value_vars = ['doggo','floofer','pupper','puppo'],
                  var_name = 'stage',
                  value_name = 'dog_stage')

# Drop stage column
df_clean.drop('stage', axis=1, inplace = True)

# Remove duplicates
df_clean = df_clean.sort_values('dog_stage').drop_duplicates(subset='tweet_id', keep='last')

### 2.9.2 Test (aggregation of type of dog columns)

In [ ]:
# Values in dog_stage column

df_clean.dog_stage.sort_values().unique()

### 2.10.1 Code (replacement of non-names)

In [ ]:
# Replace non-names with 'None'

df_clean.dog_name = df_clean.dog_name.str.replace('^[a-z]{1,}', 'None')

### 2.10.2 Test (replacement of non-names)

In [ ]:
# Visual assessment of effectiveness of RegEx .replace code

df_clean.dog_name.sort_values().unique()

### 2.11.1 Code (standardization of fraction digits in CI_prediction column

In [ ]:
df_clean = df_clean.round({"CI_prediction":6})

### 2.11.2 Test (standardization of fraction digits in CI_prediction column

In [ ]:
df_clean.CI_prediction.sample(10)

### 2.12.1 Code (Rearrangement of columns)

In [ ]:
# Change order of columns (move dog_stage from end to after dog_name)

df_clean = df_clean[['tweet_id', 'timestamp', 'source', 'text', 'rating_numerator',
       'rating_denominator', 'dog_name', 'dog_stage', 'prediction', 'CI_prediction',
       'is_dog', 'favorite_count', 'retweet_count']]

### 2.12.2 Test (Rearrangement of columns)

In [ ]:
# Check column order after preceding operation

df_clean.columns.values

### 2.13 Export dataframe to .csv

In [ ]:
df_clean.to_csv('twitter_archive_master.csv', index=False)

# 3. Analyzing and visualizing the data

## 3.1.1 Insight #1
What are the 5 most common breeds as identified by the neural network and their absolute frequency?

N.b. Only original tweets and based solely on the 1st prediciton.

**Conclusion**: The two most popular dog breeds are closely related breeds of the same family: retrievers, a breed known for its natural affection and toleration of children. The subsequent three (pembroke, chihuahua, and pug) are all small dog breeds with fairly high meme potential. By not setting any parameters, it becomes visible that (assuming that the neural network is at least fairly accurate) not all photos actually contain dogs, but it would require manual examination to confirm this hypothesis.

In [ ]:
df_clean.prediction.value_counts()

## 3.1.2 Insight #2

What is the relative frequency of (identified) dog stages?

**Conclusion**: Although nearly 85% of all dogs are uncategorized, the relative frequency of identified dogs are as following (using Lucid Software's explanation of dog-related lingo in their video "What is a Pupper? What is a Doggo?"): The most popular type of floofer (which can be any kind of dog, but usually refers to big dogs with a lot of fur) by a landslide is a smoll doggo, a so-called pupper. A big pupper, commonly known as doggo, is the second most common type of floofer. The intermediate stage between a pupper and a doggo makes up around 1% of all floofers, whereas the authentic fur-heavy floofer makes up less than half a percent of all floofers.

In [ ]:
df_clean.dog_stage.value_counts(normalize=True)

## 3.1.3 Insight #3

What are the 10 most popular dog names?

**Conclusion**: Aside from the list of the ten most popular dog names, included are some other statistics that reveal some interesting characteristics: there are (2094-704) 1390 registered dog names in this dataset. 930 of those are unique. Considering that only the 10 most popular names already account for 84 observations, the majority of dog names appear only once in this dataset. Additionally, the most frequently used names, Lucy and Charlie, still make up roughly ((22/2094)*100) 1% of all observations, confirming an incredible diversity in dog names!

In [ ]:
name = []
 
count = 0
 
for i in df_clean.dog_name:
    if i not in name:
        count += 1
        name.append(i)
        
print("The number of unique names is:\n", count)
print("\nThe 10 most popular names are:\n",df_clean.dog_name.value_counts()[:10])
print("\nThe total amount of entries is:\n", df_clean.dog_name.count())

## 3.1.4 Insight #4

How is the retweet count distributed?

**Conclusion**: As can be derived from the statistics and figure below, this variable is strongly positively skewed and leptokurtic: Nearly 75% of the data falls below the mean, the max is almost 17 standard deviations above the mean, and the IQR is only half a standard deviation. Translated to reality, out of 2079 original tweets, nearly 2,000 got less than 5,000 retweets. A few tweets went viral, leading to a max of nearly 70,000 retweets.

In [ ]:
plt.hist(df_clean.retweet_count)
plt.title('Retweet distribution')
plt.xlabel('Retweet count')
plt.ylabel('Count');

In [ ]:
df_clean.retweet_count.describe()

## 3.2 Dog stage and tweet virality visualization

Is there a correlation between the amount of times a tweet is favorited and retweeted and does this correlation differ between dog stages?

**Conclusion**: Juxtaposed with the occurrence of identified dog stages, where pupper was the most frequently occurring, in terms of retweets and favorite count, puppers are outperformed by every other dog stage. Possibly, the owners of the Twitter account WeRateDogs like puppers more than the audience does. Maybe the audience also loves puppers, but do not feel comfortable associating themselves with this type of content. Taking in consideration the dog_stage distribution, perhaps people have grown tired from seeing an overabundance of tweets with photos of puppers.

In [ ]:
# Scatterplot of retweets vs favorite count

sns.lmplot(data=df_clean,
           x="retweet_count", 
           y="favorite_count", 
           height = 6,
           aspect=2,
           hue="dog_stage",
           scatter_kws={'alpha':0.1});

plt.title('Favorite vs. retweet count by dog stage');
plt.xlabel('Retweet count');
plt.ylabel('Favorite count');